In [1]:
import os # Necessário para verificar se os arquivos existem
import pandas as pd
from icalendar import Calendar
from datetime import datetime, date, time
from plotly import figure_factory as ff
from icalendar import Calendar
import datetime
import plotly.io as pio
pio.renderers.default = "notebook"
import requests
import os
import pandas as pd

In [2]:
def ler_calendario_ics(arquivo_ics):
    """
    Lê um arquivo de calendário ICS e converte para DataFrame
    
    Args:
        arquivo_ics (str): Caminho para o arquivo .ics
        
    Returns:
        pandas.DataFrame: DataFrame com as colunas [Start, End, Summary]
    """
    try:
        # Lê o arquivo ICS
        with open(arquivo_ics, 'rb') as f:
            cal = Calendar.from_ical(f.read())
            
        # Lista para armazenar os eventos
        eventos = []
        
        # Processa cada evento do calendário
        for event in cal.walk('VEVENT'):
            try:
                # Extrai data/hora de início e fim
                start = event.get('dtstart').dt
                end = event.get('dtend').dt
                summary = str(event.get('summary'))
                
                # Converte para datetime se for apenas date
                if isinstance(start, date) and not isinstance(start, datetime):
                    start = datetime.combine(start, time(hour=16))
                if isinstance(end, date) and not isinstance(end, datetime):
                    end = datetime.combine(end, time(hour=11))
                
                # Adiciona à lista de eventos
                eventos.append({
                    'Start': start,
                    'End': end,
                    'Summary': summary
                })
            except Exception as e:
                print(f"Aviso: Pulando evento com erro: {e}")
                continue
        
        # Cria o DataFrame
        if not eventos:
            print("Nenhum evento válido encontrado no arquivo.")
            return None
            
        df = pd.DataFrame(eventos)
        
        # Ordena por data de início
        df = df.sort_values('Start')
        
        return df
    
    except FileNotFoundError:
        print(f"Erro: Arquivo '{arquivo_ics}' não encontrado.")
        return None
    except Exception as e:
        print(f"Erro ao processar o arquivo: {str(e)}")
        return None

In [3]:
def merge_ical_files(file1_path, file2_path, output_path):
    """
    Mescla dois arquivos iCal em um, evitando duplicatas baseadas
    em DTSTART, DTEND e SUMMARY. Aplica regras de tempo e resumo.
    Aceita file2_path como None.

    Args:
        file1_path (str): Caminho para o primeiro arquivo iCal (obrigatório).
        file2_path (str or None): Caminho para o segundo arquivo iCal, ou None.
        output_path (str): Caminho onde o calendário mesclado será salvo.
    """
    # Get current date and date 1 year from now
    current_date = datetime.datetime.now()
    one_year_from_now = (current_date + datetime.timedelta(days=364)).replace(tzinfo=None)

    added_events = set()
    merged_cal = Calendar()
    merged_cal.add('prodid', '-//Merged Calendar//EN')
    merged_cal.add('version', '2.0')

    # --- CORREÇÃO: Filtrar a lista para remover None ---
    # Cria a lista apenas com os caminhos que NÃO são None
    files_to_process = [p for p in [file1_path, file2_path] if p is not None]
    # ---------------------------------------------------

    if not files_to_process:
        print("  ERRO: Nenhum arquivo de entrada válido fornecido.")
        return # Sai se não houver arquivos para processar

    for file_path in files_to_process:
        # O resto do loop continua como na versão anterior, pois file_path nunca será None aqui
        print(f"  Processando arquivo: {os.path.basename(file_path)}")
        try:
            with open(file_path, 'rb') as f:
                cal = Calendar.from_ical(f.read())

            for component in cal.walk('VEVENT'):
                try:
                    start_prop = component.get('dtstart')
                    end_prop = component.get('dtend')

                    if not start_prop or not end_prop:
                        print(f"    Aviso: Pulando evento sem dtstart ou dtend: {component.get('summary')}")
                        continue

                    start = start_prop.dt
                    end = end_prop.dt
                    summary_obj = component.get('summary')
                    summary = str(summary_obj) if summary_obj else ""

                    start_compare = start
                    if hasattr(start_compare, 'tzinfo') and start_compare.tzinfo is not None:
                         start_compare = start_compare.replace(tzinfo=None)

                    is_date_only_start = isinstance(start, datetime.date) and not isinstance(start, datetime.datetime)

                    if is_date_only_start:
                        if start > one_year_from_now.date():
                            continue
                    elif start_compare > one_year_from_now:
                        continue

                    original_summary = summary
                    if summary == 'CLOSED - Not available':
                        component['summary'] = 'Booking'
                        summary = 'Booking'
                    elif summary == 'Airbnb (Not available)':
                        component['summary'] = 'Direto'
                        summary = 'Direto'
                    elif summary == 'Reserved':
                         component['summary'] = 'Airbnb'
                         summary = 'Airbnb'
                    elif not summary:
                         component['summary'] = 'Origem Desconhecida'
                         summary = 'Origem Desconhecida'

                    if is_date_only_start:
                        start = datetime.datetime.combine(start, datetime.time(hour=16, minute=0))
                        component.pop('dtstart')
                        component.add('dtstart', start)

                    is_date_only_end = isinstance(end, datetime.date) and not isinstance(end, datetime.datetime)
                    if is_date_only_end:
                         end = datetime.datetime.combine(end, datetime.time(hour=11, minute=0))
                         component.pop('dtend')
                         component.add('dtend', end)

                    start_key = component.get('dtstart').dt
                    end_key = component.get('dtend').dt
                    event_key = (start_key, end_key, summary)

                    if event_key not in added_events:
                        merged_cal.add_component(component)
                        added_events.add(event_key)

                except Exception as e_event:
                     print(f"    ERRO ao processar um evento: {e_event}. Evento: {component.get('summary')}")

        except FileNotFoundError:
             print(f"  ERRO: Arquivo '{file_path}' não encontrado.")
        except Exception as e_file:
             print(f"  ERRO: Falha ao ler o arquivo iCal '{file_path}': {e_file}")

    # Save the merged calendar
    try:
        with open(output_path, 'wb') as f:
            f.write(merged_cal.to_ical())
        print(f"  Calendário mesclado salvo com sucesso em {output_path} ({len(added_events)} eventos únicos)")
    except Exception as e_save:
        print(f"  ERRO ao salvar o arquivo mesclado '{output_path}': {e_save}")

In [4]:
# --- 1. Define apartment configurations ---
apartment_configs = {
    'c108': {'has_booking': True},
    'd014': {'has_booking': True},
    'cbl004': {'has_booking': True},
    'ap101': {'has_booking': False},  # Airbnb only
    'ap201': {'has_booking': False}   # Airbnb only
}

print("Iniciando a mesclagem dos calendários...")

# --- 2. Loop para processar todos os apartamentos com booking e airbnb ---
for apt, config in apartment_configs.items():
    print(f"\nProcessando: {apt.upper()}")
    
    # Define os nomes dos arquivos
    file_airbnb = f'calendars/{apt}_airbnb.ics'
    if config['has_booking']:
        file_booking = f'calendars/{apt}_booking.ics'
    else:
        file_booking = None
    file_merged = f'calendars/{apt}_merged_booking_airbnb.ics'
   
    merge_ical_files(
        file_airbnb,
        file_booking,
        file_merged
    )



Iniciando a mesclagem dos calendários...

Processando: C108
  Processando arquivo: c108_airbnb.ics
  Processando arquivo: c108_booking.ics
  Calendário mesclado salvo com sucesso em calendars/c108_merged_booking_airbnb.ics (12 eventos únicos)

Processando: D014
  Processando arquivo: d014_airbnb.ics
  Processando arquivo: d014_booking.ics
  Calendário mesclado salvo com sucesso em calendars/d014_merged_booking_airbnb.ics (6 eventos únicos)

Processando: CBL004
  Processando arquivo: cbl004_airbnb.ics
  Processando arquivo: cbl004_booking.ics
  Calendário mesclado salvo com sucesso em calendars/cbl004_merged_booking_airbnb.ics (16 eventos únicos)

Processando: AP101
  Processando arquivo: ap101_airbnb.ics
  Calendário mesclado salvo com sucesso em calendars/ap101_merged_booking_airbnb.ics (21 eventos únicos)

Processando: AP201
  Processando arquivo: ap201_airbnb.ics
  Calendário mesclado salvo com sucesso em calendars/ap201_merged_booking_airbnb.ics (23 eventos únicos)


In [5]:
# --- 3. Loop para processar todos os apartamentos com o google---
for apt, config in apartment_configs.items():
    print(f"\nProcessando: {apt.upper()}")
    
    # Define os nomes dos arquivos
    file_google = f'calendars/{apt}_google.ics'
    file_otas=f'calendars/{apt}_merged_booking_airbnb.ics'
    file_merged = f'calendars/{apt}_merged_booking_airbnb_google.ics'
    merge_ical_files(
        file_otas,
        file_google,
        file_merged
    )


print("\n--- Mesclagem de todos os calendários concluída! ---")


Processando: C108
  Processando arquivo: c108_merged_booking_airbnb.ics
  Processando arquivo: c108_google.ics
  Calendário mesclado salvo com sucesso em calendars/c108_merged_booking_airbnb_google.ics (47 eventos únicos)

Processando: D014
  Processando arquivo: d014_merged_booking_airbnb.ics
  Processando arquivo: d014_google.ics
  Calendário mesclado salvo com sucesso em calendars/d014_merged_booking_airbnb_google.ics (15 eventos únicos)

Processando: CBL004
  Processando arquivo: cbl004_merged_booking_airbnb.ics
  Processando arquivo: cbl004_google.ics
  Calendário mesclado salvo com sucesso em calendars/cbl004_merged_booking_airbnb_google.ics (371 eventos únicos)

Processando: AP101
  Processando arquivo: ap101_merged_booking_airbnb.ics
  Processando arquivo: ap101_google.ics
  Calendário mesclado salvo com sucesso em calendars/ap101_merged_booking_airbnb_google.ics (574 eventos únicos)

Processando: AP201
  Processando arquivo: ap201_merged_booking_airbnb.ics
  Processando arqui